In [1]:

import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
from sklearn.model_selection import KFold,cross_val_score
%matplotlib inline

from sklearn.metrics import mean_squared_error

from sklearn import model_selection, preprocessing, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import lightgbm as lgb

from sklearn.ensemble import  RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge,RidgeCV,LarsCV,LassoCV,ElasticNetCV,LassoLarsCV
#from pystacknet.pystacknet import StackNetRegressor

In [2]:
train = pd.read_excel('input/Data_Train.xlsx')
test = pd.read_excel('input/Data_Test.xlsx')

print(train.shape,test.shape)

(12690, 9) (4231, 8)


In [3]:
train.head()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49 votes,1200
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30 votes,1500
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221 votes,800
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24 votes,800
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165 votes,300


In [4]:
#train.TITLE.value_counts()

In [5]:
test.head()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES
0,CASUAL DINING,4085,"North Indian, Chinese, Mughlai, Kebab",12noon – 12midnight (Mon-Sun),Noida,Sector 18,4.3,564 votes
1,QUICK BITES,12680,"South Indian, Fast Food, Pizza, North Indian",7am – 12:30AM (Mon-Sun),Mumbai,Grant Road,4.2,61 votes
2,CASUAL DINING,1411,"North Indian, Seafood, Biryani, Chinese",11am – 11:30pm (Mon-Sun),Mumbai,Marine Lines,3.8,350 votes
3,None,204,Biryani,"9am – 10pm (Mon, Wed, Thu, Fri, Sat, Sun), 10:...",Faridabad,NIT,3.8,1445 votes
4,QUICK BITES,13453,"South Indian, Kerala",11am – 10pm (Mon-Sun),Kochi,Kaloor,3.6,23 votes


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 1), lowercase=False)
train_route = tf.fit_transform(train['TITLE'])
test_route = tf.transform(test['TITLE'])

train_route = pd.DataFrame(data=train_route.toarray(), columns=tf.get_feature_names())
test_route = pd.DataFrame(data=test_route.toarray(), columns=tf.get_feature_names())

In [7]:
train_route.shape,test_route.shape

((12690, 30), (4231, 30))

In [8]:
train_route.sum()

BAKERY            471.514829
BAR               549.623754
BEVERAGE          175.011514
BITES            3108.106127
CAFE                2.070268
CAFÉ              772.989780
CASUAL           2927.241057
CLUB               26.852224
COCKTAIL            5.331183
CONFECTIONERY       4.629156
COURT              75.851780
DESSERT           402.309869
DHABA              45.000000
DINING           2954.002643
FINE              317.793569
FOOD               91.721114
IRANI               2.070268
KIOSK              58.926850
LOUNGE            181.051354
MEAT                2.627791
MESS               53.000000
MICROBREWERY       61.264162
None             1003.000000
PAAN                4.329635
PARLOR            402.309869
PUB               122.173260
QUICK            3108.106127
SHOP              251.839552
SWEET             111.050401
TRUCK              24.303777
dtype: float64

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 1), lowercase=False)
train_CUISINES = tf.fit_transform(train['CUISINES'])
test_CUISINES = tf.transform(test['CUISINES'])

train_CUISINES = pd.DataFrame(data=train_CUISINES.toarray(), columns=tf.get_feature_names())
test_CUISINES = pd.DataFrame(data=test_CUISINES.toarray(), columns=tf.get_feature_names())

train_CUISINES.shape,test_CUISINES.shape

((12690, 131), (4231, 131))

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 1), lowercase=False)
train_TIME = tf.fit_transform(train['TIME'])
test_TIME = tf.transform(test['TIME'])

train_TIME = pd.DataFrame(data=train_TIME.toarray(), columns=tf.get_feature_names())
test_TIME = pd.DataFrame(data=test_TIME.toarray(), columns=tf.get_feature_names())

train_TIME.shape,test_TIME.shape

((12690, 90), (4231, 90))

In [11]:
train.isnull().sum()

TITLE               0
RESTAURANT_ID       0
CUISINES            0
TIME                0
CITY              112
LOCALITY           98
RATING              2
VOTES            1204
COST                0
dtype: int64

In [12]:
test.isnull().sum()

TITLE              0
RESTAURANT_ID      0
CUISINES           0
TIME               0
CITY              35
LOCALITY          30
RATING             2
VOTES            402
dtype: int64

In [13]:
train['RATING'].fillna('NEW',inplace=True)
test['RATING'].fillna('NEW',inplace=True)
train['CITY'].fillna('None',inplace=True)
test['CITY'].fillna('None',inplace=True)

train['LOCALITY'].fillna('None',inplace=True)
test['LOCALITY'].fillna('None',inplace=True)

train['VOTES'].fillna('-9999 votes',inplace=True)
test['VOTES'].fillna('-9999 votes',inplace=True)

In [14]:
train['RATING'] = train['RATING'].apply(lambda x : 0.00001 if (x == 'NEW') | (x == '-') else float(x))
test['RATING'] = test['RATING'].apply(lambda x : 0.00001 if (x == 'NEW') | (x == '-') else float(x))

In [15]:
train['votes'] = train['VOTES'].apply(lambda x : int(x.split()[0]))
test['votes'] = test['VOTES'].apply(lambda x : int(x.split()[0]))

In [16]:
train.drop('VOTES',axis=1,inplace=True)
test.drop('VOTES',axis=1,inplace=True)

In [17]:
train['city_loc'] = train['CITY'] + train['LOCALITY']
test['city_loc'] = test['CITY'] + test['LOCALITY']

In [18]:
cat_cols = ['CITY','LOCALITY','city_loc']

In [19]:
# Impute 0 for missing target values
train_df = train.copy()
test_df = test.copy()

# label encode the categorical variables and convert the numerical variables to float

for col in cat_cols:
    print(col,train_df[col].nunique(),test_df[col].nunique())
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))

CITY 360 152
LOCALITY 1417 835
city_loc 1663 934


In [20]:
#  categorical features
num_aggregations = {}
for num in ["votes", "RATING"]:
    num_aggregations[num] = [ 'max', 'mean','std','median','min']

supplier_id_train = train_df.groupby('CITY').agg({**num_aggregations})
supplier_id_train.columns = pd.Index(['SID_' + e[0] + "_" + e[1].upper() for e in supplier_id_train.columns.tolist()])

supplier_id_test = test_df.groupby('CITY').agg({**num_aggregations})
supplier_id_test.columns = pd.Index(['SID_' + e[0] + "_" + e[1].upper() for e in supplier_id_test.columns.tolist()])

supplier_id_test.shape,supplier_id_train.shape

((152, 10), (360, 10))

In [21]:
train_df = pd.merge(train_df,supplier_id_train,on='CITY',how='left')
test_df = pd.merge(test_df,supplier_id_test,on='CITY',how='left')
train_df.shape,test_df.shape

((12690, 20), (4231, 19))

In [22]:
#  categorical features
num_aggregations = {}
for num in ["votes", "RATING"]:
    num_aggregations[num] = [ 'max', 'mean','std','median','min']

supplier_id_train = train_df.groupby('LOCALITY').agg({**num_aggregations})
supplier_id_train.columns = pd.Index(['SID_' + e[0] + "_" + e[1].upper() for e in supplier_id_train.columns.tolist()])

supplier_id_test = test_df.groupby('LOCALITY').agg({**num_aggregations})
supplier_id_test.columns = pd.Index(['SID_' + e[0] + "_" + e[1].upper() for e in supplier_id_test.columns.tolist()])

supplier_id_test.shape,supplier_id_train.shape

((835, 10), (1417, 10))

In [23]:
train_df = pd.merge(train_df,supplier_id_train,on='LOCALITY',how='left')
test_df = pd.merge(test_df,supplier_id_test,on='LOCALITY',how='left')
train_df.shape,test_df.shape

((12690, 30), (4231, 29))

In [24]:
train_df.head()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,COST,votes,city_loc,...,SID_votes_MAX_y,SID_votes_MEAN_y,SID_votes_STD_y,SID_votes_MEDIAN_y,SID_votes_MIN_y,SID_RATING_MAX_y,SID_RATING_MEAN_y,SID_RATING_STD_y,SID_RATING_MEDIAN_y,SID_RATING_MIN_y
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",401,328,3.6,1200,49,1897,...,532,-912.407407,3278.664670,99.0,-9999,4.0,3.281483,1.198405,3.60,0.00001
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),75,1215,4.2,1500,30,469,...,571,-335.369565,2085.208483,79.0,-9999,4.3,3.419566,0.836026,3.60,0.00001
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",75,1273,3.8,800,221,474,...,221,113.142857,73.777658,91.0,27,3.8,3.442857,0.390969,3.50,2.70000
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),277,161,4.1,800,24,1246,...,2883,-553.446429,3364.887457,211.5,-9999,4.6,3.628573,1.308252,4.05,0.00001
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),277,709,3.8,300,165,1308,...,5724,1044.203125,1073.743063,765.0,13,4.9,4.093750,0.331842,4.10,3.40000


In [25]:
train_df.drop(['TITLE','CUISINES','TIME'],axis=1,inplace=True)
test_df.drop(['TITLE','CUISINES','TIME'],axis=1,inplace=True)

In [26]:
train_df = pd.concat([train_df,train_route,train_TIME,train_CUISINES],axis=1)
test_df = pd.concat([test_df,test_route,test_TIME,test_CUISINES],axis=1)

train_df.shape,test_df.shape

((12690, 278), (4231, 277))

In [27]:
for col1, col2 in [
                   ["votes", "RATING"],
                   
                  ]:
    train_df[col1+"_ratio_"+col2] = train_df[col1] / train_df[col2]
    test_df[col1+"_ratio_"+col2] = test_df[col1] / test_df[col2]
    train_df[col1+"_mul_"+col2] = train_df[col1] * train_df[col2]
    test_df[col1+"_mul_"+col2] = test_df[col1] * test_df[col2]

In [28]:
train_df['id'] = range(train_df.shape[0])
test_df['id'] = range(train_df.shape[0],train_df.shape[0] + test_df.shape[0])

In [29]:
for col in [
            "RESTAURANT_ID", ["CITY", "LOCALITY"],
            ["RESTAURANT_ID", "CITY"],
            ["RESTAURANT_ID", "LOCALITY"],
            ["RESTAURANT_ID", "city_loc"],
           ]:
    if not isinstance(col, list):
        col = [col]
    col_name = "_".join(col)
    all_df = pd.concat([train_df[["id"]+ col], test_df[["id"]+ col]])
    gdf = all_df.groupby(col)["id"].count().reset_index()
    gdf.columns = col + [col_name+"_count"]
    train_df = pd.merge(train_df, gdf, on=col, how="left")
    test_df = pd.merge(test_df, gdf, on=col, how="left")

In [30]:
for col in [
            "RESTAURANT_ID", ["CITY", "LOCALITY"],
            ["RESTAURANT_ID", "CITY"],
            ["RESTAURANT_ID", "LOCALITY"],
            ["RESTAURANT_ID", "city_loc"],
           ]:
    if not isinstance(col, list):
        col = [col]
    col_name = "_".join(col)
    all_df = pd.concat([train_df[["votes"]+ col], test_df[["votes"]+ col]])
    gdf = all_df.groupby(col)["votes"].mean().reset_index()
    gdf.columns = col + [col_name+"_vmean"]
    train_df = pd.merge(train_df, gdf, on=col, how="left")
    test_df = pd.merge(test_df, gdf, on=col, how="left")

In [31]:
train_df.drop('id',axis=1,inplace=True)
test_df.drop('id',axis=1,inplace=True)

In [32]:
train_y = np.log1p(train_df['COST']).values
X = train_df.drop('COST',axis=1)
y=np.log1p(train_df['COST'])
X_test = test_df.copy()

In [33]:
X.shape,X_test.shape

((12690, 289), (4231, 289))

In [34]:
set(X.columns) - set(X_test.columns)

set()

In [35]:
dev_X, val_X, dev_y, val_y = model_selection.train_test_split(X, y, test_size=0.25, random_state=42)

In [36]:
random_seed = 2019
k = 5
fold = list(KFold(k, shuffle = True, random_state = random_seed).split(train))
np.random.seed(random_seed)
from datetime import datetime

In [37]:
print(dev_X.shape,val_X.shape,test.shape,val_y.shape,dev_y.shape)

(9517, 289) (3173, 289) (4231, 9) (3173,) (9517,)


In [38]:
# custom function to run light gbm model
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return pred_test_y, model, pred_val_y

In [39]:
# Training the model #
pred_test, model, pred_val = run_lgb(dev_X, dev_y, val_X, val_y, X_test)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.358373
[200]	valid_0's rmse: 0.357047
Early stopping, best iteration is:
[161]	valid_0's rmse: 0.356678


In [41]:
pred_test[pred_test<0] =0 

In [42]:
test = pd.read_excel('input/Data_Test.xlsx')
test['COST'] = np.round(np.expm1(pred_test))

from pandas import ExcelWriter
writer = ExcelWriter('MH_v3_lgb.xlsx')
test[['COST']].to_excel(writer,'Sheet1')
writer.save()